In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

In [10]:
data_path = './data'
ycol = 'IS_FLAG'
train_archive = pd.read_csv(os.path.join(data_path, 'training_dataset/训练组_比特币挖矿_档案明细（20211220）.csv'), encoding='gb18030', index_col=0)
train_monthly_electricity = pd.read_csv(os.path.join(data_path, 'training_dataset/训练组_比特币挖矿_月用电明细（20211217）.csv'))
test_archive = pd.read_csv(os.path.join(data_path, 'test_dataset/测试组_比特币挖矿_档案明细（20211220）.csv'), encoding='gb18030')
test_monthly_electricity = pd.read_csv(os.path.join(data_path, 'test_dataset/测试组_比特币挖矿_月用电明细（20211217）.csv'))

archive = pd.concat([train_archive, test_archive], axis=0, ignore_index=True)
monthly_electricity = pd.concat([train_monthly_electricity, test_monthly_electricity], axis=0, ignore_index=True)

In [14]:
# 挖矿行为的主要特点是耗电量较高, 且峰电量, 谷电量, 平电量的数据比较均衡。
pq_col = ['pq_f', 'pq_g', 'pq_p']
monthly_electricity['pq_cv'] = monthly_electricity[pq_col].std(axis=1) / (monthly_electricity[pq_col].mean(axis=1) + 1e-5)
pq = monthly_electricity.groupby('id').agg({'pq_z':'mean', 'pq_cv':'mean'}).reset_index()
pq.columns = ['ID', 'pq_z', 'pq_cv']
pq

,ID,pq_z,pq_cv
0,179404030,1592.772727,0.912731
1,179404040,508.590909,1.732051
2,179404046,2620.000000,0.672467
3,179404049,1524.318182,0.969545
4,179404054,450.136364,1.732051
...,...,...,...
24214,2878310631,102.772727,1.123693
24215,2878334859,155.590909,0.954452
24216,2880078775,667.545455,1.062154
24217,2880249132,587.000000,1.537137


In [ ]:
data = pd.merge(archive, pq, on='ID', how='left')
train = data[~data[ycol].isnull()].copy()
test = data[data[ycol].isnull()].copy()

In [15]:
train['pre'] = 0
train.loc[(train['pq_cv']<0.085) & (train['pq_z']>4500), 'pre'] = 1
print(f"训练集预测 {sum(train['pre'])} 个 1")
valid_f1_score = f1_score(train[ycol], train['pre'], average='macro')
print(f"训练集 f1: {valid_f1_score:.6f}")

训练集预测 13 个 1
训练集 f1: 0.847628


In [6]:
test[ycol] = 0
test.loc[(test['pq_cv']<0.085) & (test['pq_z']>4500), ycol] = 1
test[['ID', ycol]].rename(columns={'ID': 'id', ycol: 'label'}).to_csv(f'./results/submit.csv', index=False)
print(f"测试集预测 {sum(test[ycol])} 个 1")

测试集预测 44 个 1
